In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori

In [2]:
dataset = pd.read_csv('Sujet/bank-data.csv')

## Preprocessing on age

In [3]:
age1020 = dataset.loc[(dataset['age'] > 10) & (dataset['age'] <= 20),:]
age1020.loc[:,'age'] = "10-20"
age2030 = dataset.loc[(dataset['age'] > 20) & (dataset['age'] <= 30),:]
age2030.loc[:,'age'] = "20-30"
age3040 = dataset.loc[(dataset['age'] > 30) & (dataset['age'] <= 40),:]
age3040.loc[:,'age'] = "30-40"
age4050 = dataset.loc[(dataset['age'] > 40) & (dataset['age'] <= 50),:]
age4050.loc[:,'age'] = "40-50"
age5060 = dataset.loc[(dataset['age'] > 50) & (dataset['age'] <= 60),:]
age5060.loc[:,'age'] = "50-60"
age6070 = dataset.loc[(dataset['age'] > 60) & (dataset['age'] <= 70),:]
age6070.loc[:,'age'] = "60-70"
dataset = pd.concat ([age1020, age2030, age3040, age4050, age5060, age6070])

/Users/jouffroy/.local/share/virtualenvs/ProjetUdeS-ExtractionRegles-o6pjDpBg/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Preprocessing on income

In [4]:
print(dataset['income'].max())
print(dataset['income'].min())

63130.1
5014.21


In [5]:
income010k = dataset.loc[(dataset['income'] > 0) & (dataset['income'] <= 10000),:]
income010k.loc[:,'income'] = "0-10k"
income10k20k = dataset.loc[(dataset['income'] > 10000) & (dataset['income'] <= 20000),:]
income10k20k.loc[:,'income'] = "10k-20k"
income20k30k = dataset.loc[(dataset['income'] > 20000) & (dataset['income'] <= 30000),:]
income20k30k.loc[:,'income'] = "20k-30k"
income30k40k = dataset.loc[(dataset['income'] > 30000) & (dataset['income'] <= 40000),:]
income30k40k.loc[:,'income'] = "30k-40k"
income40k50k = dataset.loc[(dataset['income'] > 40000) & (dataset['income'] <= 50000),:]
income40k50k.loc[:,'income'] = "40k-50k"
income50k60k = dataset.loc[(dataset['income'] > 50000) & (dataset['income'] <= 60000),:]
income50k60k.loc[:,'income'] = "50k-60k"
income60k70k = dataset.loc[(dataset['income'] > 60000) & (dataset['income'] <= 70000),:]
income60k70k.loc[:,'income'] = "60k-70k"

dataset = pd.concat ([income010k, income10k20k, income20k30k, income30k40k, income40k50k, income50k60k, income60k70k])

## Preprocessing on children

In [6]:
print(dataset['children'].min())
print(dataset['children'].max())
dataset.astype('str')

0
3


,id,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
95,ID12196,10-20,FEMALE,INNER_CITY,0-10k,YES,0,NO,YES,YES,NO,NO
155,ID12256,10-20,FEMALE,SUBURBAN,0-10k,YES,2,NO,YES,YES,NO,YES
166,ID12267,10-20,MALE,TOWN,0-10k,NO,1,NO,YES,YES,YES,NO
275,ID12376,10-20,MALE,RURAL,0-10k,YES,0,YES,NO,YES,YES,YES
297,ID12398,10-20,MALE,RURAL,0-10k,YES,2,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
580,ID12681,60-70,MALE,SUBURBAN,50k-60k,YES,2,YES,YES,NO,YES,YES
593,ID12694,60-70,MALE,SUBURBAN,50k-60k,YES,2,NO,YES,YES,NO,YES
190,ID12291,60-70,FEMALE,SUBURBAN,60k-70k,YES,2,YES,YES,YES,NO,YES
270,ID12371,60-70,FEMALE,SUBURBAN,60k-70k,NO,2,NO,YES,YES,YES,YES


## Preprocessing on married

In [7]:
nomarried = dataset.loc[dataset['married'] == "NO",:]
nomarried.loc[:,'married'] = "nomarried"
yesmarried = dataset.loc[dataset['married'] == "YES",:]
yesmarried.loc[:,'married'] = "yesmarried"

dataset = pd.concat([nomarried, yesmarried])

## Preprocessing on car

In [8]:
nocar = dataset.loc[dataset['car'] == "NO",:]
nocar.loc[:,'car'] = "nocar"
yescar = dataset.loc[dataset['car'] == "YES",:]
yescar.loc[:,'car'] = "yescar"

dataset = pd.concat([nocar, yescar])

## Preprocessing on save_act

In [9]:
nosave_act = dataset.loc[dataset['save_act'] == "NO",:]
nosave_act.loc[:,'save_act'] = "no_save_act"
yessave_act = dataset.loc[dataset['save_act'] == "YES",:]
yessave_act.loc[:,'save_act'] = "yes_save_act"

dataset = pd.concat([nosave_act, yessave_act])

## Preprocessing on current_act

In [10]:
nocurrent_act = dataset.loc[dataset['current_act'] == "NO",:]
nocurrent_act.loc[:,'current_act'] = "no_current_act"
yescurrent_act = dataset.loc[dataset['current_act'] == "YES",:]
yescurrent_act.loc[:,'current_act'] = "yes_current_act"

dataset = pd.concat([nocurrent_act, yescurrent_act])

## Preprocessing on mortgage

In [11]:
nomortgage = dataset.loc[dataset['mortgage'] == "NO",:]
nomortgage.loc[:,'mortgage'] = "nomortgage"
yesmortgage = dataset.loc[dataset['mortgage'] == "YES",:]
yesmortgage.loc[:,'mortgage'] = "yesmortgage"

dataset = pd.concat([nomortgage, yesmortgage])

## Preprocessing on pep

In [12]:
nopep = dataset.loc[dataset['pep'] == "NO",:]
nopep.loc[:,'pep'] = "nopep"
yespep = dataset.loc[dataset['pep'] == "YES",:]
yespep.loc[:,'pep'] = "yespep"

dataset = pd.concat([nopep, yespep])
dataset.head()

,id,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
259,ID12360,30-40,MALE,TOWN,10k-20k,nomarried,0,nocar,no_save_act,no_current_act,nomortgage,nopep
297,ID12398,10-20,MALE,RURAL,0-10k,yesmarried,2,nocar,no_save_act,no_current_act,nomortgage,nopep
285,ID12386,20-30,MALE,TOWN,0-10k,yesmarried,0,nocar,no_save_act,no_current_act,nomortgage,nopep
161,ID12262,30-40,FEMALE,INNER_CITY,10k-20k,yesmarried,0,nocar,no_save_act,no_current_act,nomortgage,nopep
590,ID12691,20-30,MALE,INNER_CITY,20k-30k,yesmarried,0,nocar,no_save_act,no_current_act,nomortgage,nopep


## Separate epargnant / non epargnant

In [13]:
epargnant_pep = dataset[((dataset['pep'] == "yespep") & (dataset['mortgage'] == "nomortgage")) ]
epargnant_mortgage = dataset[((dataset['pep'] == "nopep") & (dataset['mortgage'] == "yesmortgage"))]
epargnant_both =dataset[((dataset['pep'] == "yespep") & (dataset['mortgage'] == "yesmortgage"))]
non_epargnant = dataset[((dataset['pep'] == "nopep") & (dataset['mortgage'] == "nomortgage")) ]
epargnant = pd.concat([epargnant_pep, epargnant_mortgage, epargnant_both])
epargnant.head()

,id,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
372,ID12473,20-30,FEMALE,INNER_CITY,0-10k,nomarried,0,nocar,no_save_act,no_current_act,nomortgage,yespep
0,ID12101,40-50,FEMALE,INNER_CITY,10k-20k,nomarried,1,nocar,no_save_act,no_current_act,nomortgage,yespep
178,ID12279,40-50,MALE,INNER_CITY,10k-20k,yesmarried,0,nocar,no_save_act,no_current_act,nomortgage,yespep
253,ID12354,40-50,FEMALE,TOWN,10k-20k,yesmarried,1,nocar,no_save_act,no_current_act,nomortgage,yespep
280,ID12381,30-40,MALE,INNER_CITY,20k-30k,yesmarried,1,nocar,no_save_act,no_current_act,nomortgage,yespep


## Apriori algorithm

In [79]:
items = []
for i in range(0, len(epargnant)):
    items.append([str(epargnant.values[i,j]) for j in range(0, len(epargnant.columns))])

In [85]:
association_rules = apriori(items, min_support=0.45,  min_lift=0.5, min_confidence=0.2, min_length=3)

In [86]:
association_results = list(association_rules)

In [87]:
print(len(association_results))

14


In [78]:
for item in association_results:
    print("Rule: " + str(list(item[2][0][0])) + " -> " + str(list(item[2][0][1])))
    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: [] -> ['MALE']
Support: 0.5268542199488491
Confidence: 0.5268542199488491
Lift: 1.0
Rule: [] -> ['nocar']
Support: 0.5012787723785166
Confidence: 0.5012787723785166
Lift: 1.0
Rule: [] -> ['yes_current_act']
Support: 0.7595907928388747
Confidence: 0.7595907928388747
Lift: 1.0
Rule: [] -> ['yes_save_act']
Support: 0.6956521739130435
Confidence: 0.6956521739130435
Lift: 1.0
Rule: [] -> ['yesmarried']
Support: 0.5754475703324808
Confidence: 0.5754475703324808
Lift: 1.0
Rule: [] -> ['yesmortgage']
Support: 0.5345268542199488
Confidence: 0.5345268542199488
Lift: 1.0
Rule: [] -> ['yespep']
Support: 0.7007672634271099
Confidence: 0.7007672634271099
Lift: 1.0
Rule: ['yes_current_act'] -> ['yes_save_act']
Support: 0.5396419437340153
Confidence: 0.7104377104377104
Lift: 1.0212542087542087
Rule: ['yes_current_act'] -> ['yespep']
Support: 0.5396419437340153
Confidence: 0.7104377104377104
Lift: 1.0137997984713314
